In [84]:
import numpy as np
import matplotlib.pyplot as plt
import torch

Predictin tempretures:
\begin{align} 
    t_c &: celsius \\
    t_u &: unknown units
\end{align}

In [85]:
# Data
t_c = [0.5, 14.0, 15.0, 28.0, 11.0, 8.0, 3.0, -4.0, 6.0, 13.0, 21.0]
t_u = [35.7, 55.9, 58.2, 81.9, 56.3, 48.9, 33.9, 21.8, 48.4, 60.4, 68.4]

In [86]:
t_c = torch.tensor(t_c)
t_u = torch.tensor(t_u)

In [87]:
def model(t_u, w, b):
    return w * t_u + b

def loss_f(t_p, t_c):
    squared_difference = (t_p - t_c)**2
    return squared_difference.mean()

def gradient(alpha, t_p, t_c, t_u, w, b):
    dw = 2 * ((t_p - t_c) * t_u).mean()
    db = 2 * (t_p - t_c).mean()
    
    w = w - alpha * dw
    b = b - alpha * db
    return w, b


In [88]:
w = torch.ones(())
b = torch.zeros(())

In [125]:
epoch = 100

for i in range(epoch):
    t_p = model(t_u, w, b)
    loss = loss_f(t_p, t_c)
    w, b = gradient(0.0001, t_p, t_c, t_u, w , b)
    print("Epoch", i+1, loss.item())


Epoch 1 29.01170539855957
Epoch 2 29.01078987121582
Epoch 3 29.009876251220703
Epoch 4 29.00896453857422
Epoch 5 29.00804901123047
Epoch 6 29.007139205932617
Epoch 7 29.0062198638916
Epoch 8 29.005311965942383
Epoch 9 29.00439453125
Epoch 10 29.00348663330078
Epoch 11 29.0025691986084
Epoch 12 29.00165557861328
Epoch 13 29.00074577331543
Epoch 14 28.999834060668945
Epoch 15 28.998920440673828
Epoch 16 28.998008728027344
Epoch 17 28.997093200683594
Epoch 18 28.996179580688477
Epoch 19 28.995264053344727
Epoch 20 28.994352340698242
Epoch 21 28.993440628051758
Epoch 22 28.992523193359375
Epoch 23 28.991615295410156
Epoch 24 28.990697860717773
Epoch 25 28.989789962768555
Epoch 26 28.988874435424805
Epoch 27 28.987958908081055
Epoch 28 28.9870548248291
Epoch 29 28.98613929748535
Epoch 30 28.9852237701416
Epoch 31 28.98431396484375
Epoch 32 28.9833984375
Epoch 33 28.982484817504883
Epoch 34 28.98157501220703
Epoch 35 28.980663299560547
Epoch 36 28.979753494262695
Epoch 37 28.978837966918945
